# Try a new method

[link to their github repo](https://github.com/ZhengzeZhou/unbiased-feature-importance)

In [1]:
import shap
import pandas as pd
import scipy as sp
import numpy as np
from irf.ensemble import wrf
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import matplotlib.pyplot as plt
from treeinterpreter.feature_importance import feature_importance as FI
from treeinterpreter.feature_importance import MDA
import sys
sys.path += ['../code/']
from UFI_wrapper import UFI_importance

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
### load data

num = 2

problem = [0]
tree_depth = [1]
for num in range(1, 3):
    for problem in ['classification', 'regression']:
        for tree_depth in ['deep', 'shallow']:
            #if tree_depth == 'deep':
            #    continue
            if not (num == 1 and tree_depth == 'deep' and problem == 'regression'):
                continue
            print('working on {} {} {}'.format(num, problem, tree_depth))
            gini_list, ufi_list = [], []
            for i in range(40):
                print(i)
                X_train = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_X_train.csv'.format(problem, num, i), delimiter=',')
                y_train = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_y_train.csv'.format(problem, num, i), delimiter=',')
                X_test = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_X_test.csv'.format(problem, num, i), delimiter=',')
                y_test = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_y_test.csv'.format(problem, num, i), delimiter=',')
                noisy_features = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_noisy_features.csv'.format(problem, num, i), delimiter=',').astype(np.int)
                n_features = len(noisy_features)

                ### Train RF
                if problem == 'classification':
                    if tree_depth == 'deep':
                        rf = rfc(n_estimators=100, max_features=10, min_samples_leaf=1)
                    else:
                        rf = rfc(n_estimators=100, max_features=10, min_samples_leaf=100)
                else:
                    if tree_depth == 'deep':
                        rf = rfr(n_estimators=100, max_features= 10, min_samples_leaf=1)
                    else:
                        rf = rfr(n_estimators=100, max_features= 10, min_samples_leaf=100)
                rf.fit(X_train, y_train)
                gini_imp = rf.feature_importances_
                ufi_imp = UFI_importance(rf, X_train, y_train)
                ufi_imp[ufi_imp < 0] = 0
                gini_list.append(roc_auc_score(noisy_features, - gini_imp))
                ufi_list.append(roc_auc_score(noisy_features, - ufi_imp))
                
            np.savez('../intermediate/04_aggregate/{}{}/add_new_results_{}.npz'.format(problem, num, tree_depth), gini_list=gini_list, ufi_list=ufi_list)


working on 1 regression deep
0
1
2
3


In [27]:
problem = 'classification'
tree_depth = 'deep'
num = 2
i = 0
X_train = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_X_train.csv'.format(problem, num, i), delimiter=',')
y_train = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_y_train.csv'.format(problem, num, i), delimiter=',')
X_test = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_X_test.csv'.format(problem, num, i), delimiter=',')
y_test = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_y_test.csv'.format(problem, num, i), delimiter=',')

In [32]:
rf = rfc(n_estimators=100, max_features=10, min_samples_leaf=1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
%timeit ufi_imp = UFI_importance(rf, X_train, y_train)

In [ ]:
%timeit feature_importance(rf, )

In [22]:
np.mean(ufi_list)

0.5377777777777778

In [19]:
np.mean(gini_list)

0.6886913580246914

In [20]:
len(ufi_list)

27